# Example: DC-DC converter


We consider a boost DC-DC converter which has been widely studied from the point of view of hybrid control, see for example in  [1, V.A],[2],[3].
This is a **safety problem** for a **switching system**.

![Boost DC-DC converter.](https://github.com/dionysos-dev/Dionysos.jl/blob/master/docs/assets/dcdcboost.jpg?raw=true)

The state of the system is given by $x(t) = \begin{bmatrix} i_l(t) & v_c(t) \end{bmatrix}^\top$.
The switching system has two modes consisting in two-dimensional affine dynamics:
$$
\dot{x} = f_p(x) = A_p x + b_p,\quad p=1,2
$$
with
$$
A_1 = \begin{bmatrix} -\frac{r_l}{x_l} &0 \\ 0 & -\frac{1}{x_c}\frac{1}{r_0+r_c}  \end{bmatrix}, A_2= \begin{bmatrix} -\frac{1}{x_l}\left(r_l+\frac{r_0r_c}{r_0+r_c}\right) & -\frac{1}{x_l}\frac{r_0}{r_0+r_c}  \\ \frac{1}{x_c}\frac{r_0}{r_0+r_c}   & -\frac{1}{x_c}\frac{1}{r_0+r_c}  \end{bmatrix}, b = \begin{bmatrix} \frac{v_s}{x_l}\\0\end{bmatrix}.
$$
The goal is to design a controller to keep the state of the system in a safety region around the reference desired value, using as input only the switching
signal.


In order to study the concrete system and its symbolic abstraction in a unified framework, we will solve the problem
for the sampled system with a sampling time $\tau$.

The abstraction is based on a feedback refinment relation [4,V.2 Definition].
Basically, this is equivalent to an alternating simulation relationship with the additional constraint that the input of the
concrete and symbolic system preserving the relation must be identical.
This allows to easily determine the controller of the concrete system from the abstraction controller by simply adding a quantization step.

For the construction of the relations in the abstraction, it is necessary to over-approximate attainable sets of
a particular cell. In this example, we consider the used of a growth bound function  [4, VIII.2, VIII.5] which is one of the possible methods to over-approximate
attainable sets of a particular cell based on the state reach by its center. Therefore, it is used
to compute the relations in the abstraction based on the feedback refinement relation.

First, let us import [StaticArrays](https://github.com/JuliaArrays/StaticArrays.jl).

In [1]:
using StaticArrays

At this point, we import the useful Dionysos sub-module for this problem: [Abstraction](https://github.com/dionysos-dev/Dionysos.jl/blob/master/src/Abstraction/abstraction.jl).

In [2]:
using Dionysos
using Dionysos.Problem
const DI = Dionysos
const UT = DI.Utils
const DO = DI.Domain
const ST = DI.System
const CO = DI.Control
const SY = DI.Symbolic

Dionysos.Symbolic

### Definition of the system
we can import the module containing the DCDC problem like this

In [3]:
include(joinpath(dirname(dirname(pathof(Dionysos))), "problems", "DCDC.jl"))

Main.var"##313".DCDC

and we can instantiate the DC system with the provided system

In [4]:
problem = DCDC.problem()

x0 = SVector(0.0, 0.0)
hx = SVector(2.0/4.0e3, 2.0/4.0e3)
state_grid = DO.GridFree(x0, hx)
u0 = SVector(1)
hu = SVector(1)
input_grid = DO.GridFree(u0, hu)

using JuMP
optimizer = MOI.instantiate(Abstraction.Optimizer)
MOI.set(optimizer, MOI.RawOptimizerAttribute("problem"), problem)
MOI.set(optimizer, MOI.RawOptimizerAttribute("state_grid"), state_grid)
MOI.set(optimizer, MOI.RawOptimizerAttribute("input_grid"), input_grid)
MOI.optimize!(optimizer)

controller = MOI.get(optimizer, MOI.RawOptimizerAttribute("controller"));

compute_symmodel_from_controlsystem! started
compute_symmodel_from_controlsystem! terminated with success: 3799110 transitions created
  5.596290 seconds (47.97 M allocations: 1.814 GiB, 7.40% gc time, 7.42% compilation time)
compute_controller_safe! started

compute_controller_safe! terminated without covering init set
  5.024358 seconds (19.81 M allocations: 689.748 MiB, 3.75% gc time, 11.57% compilation time)


### Trajectory display
We choose the number of steps `nsteps` for the sampled system, i.e. the total elapsed time: `nstep`*`tstep`
as well as the true initial state `x0` which is contained in the initial state-space defined previously.

In [5]:
#nstep = 300;
#x0 = SVector(1.2, 5.6);

To complete

### References
1. A. Girard, G. Pola and P. Tabuada, "Approximately Bisimilar Symbolic Models for Incrementally Stable Switched Systems," in IEEE Transactions on Automatic Control, vol. 55, no. 1, pp. 116-126, Jan. 2010.
2. S. Mouelhi, A. Girard, and G. Gössler. “CoSyMA: a tool for controller synthesis using multi-scale abstractions”. In: HSCC. ACM. 2013, pp. 83–88.
3. A. Girard. “Controller synthesis for safety and reachability via approximate bisimulation”. In: Automatica 48.5 (2012), pp. 947–953.
4. G. Reissig, A. Weber and M. Rungger, "Feedback Refinement Relations for the Synthesis of Symbolic Controllers," in IEEE Transactions on Automatic Control, vol. 62, no. 4, pp. 1781-1796.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*